Can we learn the parameters of the $\phi^4$ theory only from correlations? Yes. Rewrite the score function in terms of moments only and learn as usual. We generate samples, then find relevant correlations, and learn from them

In [ ]:
using AdvancedHMC, ForwardDiff
using LogDensityProblems, LinearAlgebra, Random, Statistics, Optim, GalacticOptim, Plots

# Define the Phi4Model structure for 2D
struct Phi4Model2D
    lattice_size::Int  # Lattice size (NxN)
    lambda::Float64    # Quartic coupling constant
    m2 :: Float64      # Mass term squared
end

function generate_phi4_data_hmc_2d(model::Phi4Model2D, n_samples::Int, n_adapts::Int = 1000)
    lattice_size = model.lattice_size
    phi4_target = Phi4TargetDensity2D(lattice_size^2, model)

    # Define the initial 2D field configuration (NxN matrix)
    initial_phi = randn(lattice_size, lattice_size)

    # Define a Hamiltonian system
    metric = DiagEuclideanMetric(lattice_size^2)
    hamiltonian = Hamiltonian(metric, phi4_target, ForwardDiff)

    # Define a leapfrog solver with an initial step size chosen heuristically
    initial_epsilon = find_good_stepsize(hamiltonian, vec(initial_phi))
    integrator = Leapfrog(initial_epsilon)

    # Define an HMC sampler with Multinomial sampling and Generalized No-U-Turn
    kernel = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

    # Run the sampler to draw samples from the Phi4 model
    samples_vector, stats = sample(hamiltonian, kernel, vec(initial_phi), n_samples, adaptor, n_adapts; progress=false)

    # Reshape the samples back into 2D matrices
    samples = [reshape(sample, lattice_size, lattice_size) for sample in samples_vector]

    return samples
end

# Function to calculate the total energy (kinetic + potential) of a 2D field configuration
function calculate_energy_2d(phi::Matrix{T}, model::Phi4Model2D) where T<:Real
    N = size(phi, 1)
    kinetic_energy = 0.0
    potential_energy = 0.0
    # Loop through all the lattice points and calculate kinetic and potential terms
    for i in 1:N, j in 1:N
        # Kinetic term: sum over nearest neighbors (discrete Laplacian)
        i_right = mod1(i + 1, N)
        j_up = mod1(j + 1, N)
        kinetic_energy += 0.5 * ((phi[i, j] - phi[i_right, j])^2 + (phi[i, j] - phi[i, j_up])^2)
        
        # Potential term: phi^4 interaction + mass term
        potential_energy += 0.5 * model.m2 * phi[i, j]^2 + 0.25 * model.lambda * phi[i, j]^4 
    end
    
    return kinetic_energy + potential_energy
end

# Define the Phi4TargetDensity structure for 2D
struct Phi4TargetDensity2D
    dim::Int
    model::Phi4Model2D
end

# Implement the log-density function for Phi4 2D
function LogDensityProblems.logdensity(p::Phi4TargetDensity2D, phi_vector::Vector{T}) where T<:Real
    # Reshape the vector back into a 2D matrix
    N = p.model.lattice_size
    phi = reshape(phi_vector, N, N)
    return -calculate_energy_2d(phi, p.model)
end

LogDensityProblems.dimension(p::Phi4TargetDensity2D) = p.model.lattice_size^2
LogDensityProblems.capabilities(::Type{Phi4TargetDensity2D}) = LogDensityProblems.LogDensityOrder{1}()

#Function to compute moments needed for moment-based score matching in 2D
function compute_phi4_moments(data::Vector{Matrix{Float64}})
    N = size(data[1], 1)
    n_samples = length(data)
    
    # Initialize accumulators
    acc_phi2 = 0.0
    acc_phi4 = 0.0
    acc_phi6 = 0.0
    acc_phi_phi_nn = 0.0
    acc_phi3_phi_nn = 0.0
    acc_phi2_nn = 0.0
    acc_nn1_nn2 = 0.0

    # For each sample
    for sample in data
        # Loop over all sites
        for i in 1:N, j in 1:N
            phi0 = sample[i, j]

            # Nearest neighbors (all 4 directions)
            nn = [
                sample[mod1(i-1,N), j],
                sample[mod1(i+1,N), j],
                sample[i, mod1(j-1,N)],
                sample[i, mod1(j+1,N)]
            ]

            # On-site moments
            acc_phi2 += phi0^2
            acc_phi4 += phi0^4
            acc_phi6 += phi0^6

            # Site-neighbor correlations (sum over all 4 neighbors)
            for k in 1:4
                acc_phi_phi_nn += phi0 * nn[k]
                acc_phi3_phi_nn += phi0^3 * nn[k]
                acc_phi2_nn += nn[k]^2
            end

            # Neighbor-neighbor correlations (all pairs of distinct neighbors)
            for a in 1:4
                for b in (a+1):4
                    acc_nn1_nn2 += nn[a] * nn[b]
                end
            end
        end
    end

    # Normalization
    total_sites = n_samples * N^2
    
    # Each site contributes once
    mean_phi2 = acc_phi2 / total_sites
    mean_phi4 = acc_phi4 / total_sites
    mean_phi6 = acc_phi6 / total_sites
    
    # Each site contributes 4 neighbor terms
    mean_phi_phi_nn = acc_phi_phi_nn / (total_sites * 4)
    mean_phi3_phi_nn = acc_phi3_phi_nn / (total_sites * 4)
    mean_phi2_nn = acc_phi2_nn / (total_sites * 4)
    
    # Each site contributes 6 neighbor-neighbor pairs
    mean_nn1_nn2 = acc_nn1_nn2 / (total_sites * 6)

    return (
        mean_phi2, mean_phi4, mean_phi6,
        mean_phi_phi_nn, mean_phi3_phi_nn,
        mean_phi2_nn, mean_nn1_nn2
    )
end

function phi4_score_matching_loss_from_correlations(
    p::AbstractVector{<:Real},
    moments::NTuple{7,Float64},
    N::Int  # lattice size - CRITICAL PARAMETER
)
    m2, λ = p
    φ2, φ4, φ6, φφ_nn, φ3φ_nn, φ2_nn, nn1nn2 = moments
    
    # Score function expansion: s(φ) = -(4 + m2)*φ + Σnn - λ*φ³
    # E[s²] = E[(-(4+m2)*φ + Σnn - λ*φ³)²]
    
    term_A2 = (4 + m2)^2 * φ2                    # (-(4+m2)*φ)²
    term_C2 = λ^2 * φ6                          # (-λ*φ³)²
    term_B2 = 4 * φ2_nn + 12 * nn1nn2           # (Σnn)²
    term_2AB = -2 * (4 + m2) * 4 * φφ_nn        # 2*(-(4+m2)*φ)*(Σnn)
    term_2AC = 2 * λ * (4 + m2) * φ4            # 2*(-(4+m2)*φ)*(-λ*φ³)
    term_2BC = -2 * λ * 4 * φ3φ_nn              # 2*(Σnn)*(-λ*φ³)
    
    ESq = term_A2 + term_B2 + term_C2 + term_2AB + term_2AC + term_2BC
    
    # Trace term: ∂²s/∂φ² = -(m2 + 3λφ² + 4)
    trace_term = -(m2 + 4) - 3 * λ * φ2
    
    # CRITICAL: Scale by lattice volume to match direct method
    volume_factor = N^2
    return volume_factor * (0.5 * ESq + trace_term)
end

# Function to estimate the parameters λ and m² from correlations using score matching in 2D
function estimate_parameters_score_matching_moments(data::Vector{Matrix{Float64}})
    N = size(data[1], 1)
    moments = compute_phi4_moments(data)
    
    f(x) = phi4_score_matching_loss_from_correlations(x, moments, N)
    initial_x = [0.1, 0.1]  # [m2, λ]
    result = optimize(f, initial_x, ParticleSwarm())
    
    return result.minimizer[2], result.minimizer[1]  # return λ, m2
end


lattice_size = 4
lambda_0 = 2.0
m2_0 = 0.3
true_model = Phi4Model2D(lattice_size, lambda_0, m2_0)  # Use m²

# Define sample sizes
n_samples = 20000


println("Processing sample size: $n_samples")
samples = generate_phi4_data_hmc_2d(true_model, n_samples)  # HMC sampling

# Estimate parameters using score matching
λ_est, m2_est = estimate_parameters_score_matching_moments(samples)

Now code to verify if the errors from moment-based learning scale correctly as $1/\sqrt(N)$

In [ ]:
# Number of repetitions per sample size
n_trials = 20

# Pre‐compute the list of sample sizes
sample_sizes = Int[4000]
while sample_sizes[end] * 2 ≤ 2_000_000
    push!(sample_sizes, sample_sizes[end] * 2)
end

# Prepare results Dict: N → Vector of NamedTuples for each trial
results = Dict{Int, Vector{NamedTuple{(:m2, :λ, :err_m2, :err_λ),
                                      Tuple{Float64,Float64,Float64,Float64}}}}()
for N in sample_sizes
    results[N] = Vector{NamedTuple{(:m2, :λ, :err_m2, :err_λ),
                                   Tuple{Float64,Float64,Float64,Float64}}}(undef, 0)
end

# Main loop: for each N, repeat n_trials times

for N in sample_sizes
    println("→ running N = $N with $n_trials trials")
    for trial in 1:n_trials
        # 1) draw N HMC samples
        samples = generate_phi4_data_hmc_2d(true_model, N)
        # 2) estimate parameters (returns λ_est, m2_est)
        λ_est, m2_est = estimate_parameters_score_matching_moments(samples)
        # 3) compute absolute errors
        err_m2 = abs(m2_est - m2_0)
        err_λ  = abs(λ_est  - lambda_0)
        # 4) store result for this trial
        push!(results[N], (m2=m2_est, λ=λ_est, err_m2=err_m2, err_λ=err_λ))
    end
end
